In [2]:
import pandas as pd
import numpy as np
import time

In [95]:
raw_data = pd.read_csv("../Data/1m.csv")

In [96]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 41 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   Num          999999 non-null  int64
 1   Interval     999999 non-null  int64
 2   bit19        999999 non-null  int64
 3   bit18        999999 non-null  int64
 4   bit17        999999 non-null  int64
 5   bit16        999999 non-null  int64
 6   bit15        999999 non-null  int64
 7   bit14        999999 non-null  int64
 8   bit13        999999 non-null  int64
 9   bit12        999999 non-null  int64
 10  bit11        999999 non-null  int64
 11  bit10        999999 non-null  int64
 12  bit9         999999 non-null  int64
 13  bit8         999999 non-null  int64
 14  bit7         999999 non-null  int64
 15  bit6         999999 non-null  int64
 16  bit5         999999 non-null  int64
 17  bit4         999999 non-null  int64
 18  bit3         999999 non-null  int64
 19  bit2         999999 non

In [97]:
raw_data.head()

,Num,Interval,bit19,bit18,bit17,bit16,bit15,bit14,bit13,bit12,...,window9_10,window10_11,window11_12,window12_13,window13_14,window14_15,window15_16,window16_17,window17_18,window18_19
0,2,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
raw_data["Num"].values

array([       2,        3,        5, ..., 15485843, 15485849, 15485857])

In [67]:
raw_data["Num"].size

999999

In [59]:
# for i in range(19,-1,-1):
#     raw_data = raw_data.assign(**{f"bit{i}": raw_data['Num'].apply(lambda x: (x >> i)&1)})

In [72]:
# # Define the window size (e.g., 2-bit windows)
# window_size = 2

# # Iterate over the starting positions of the windows
# for i in range(20 - window_size + 1):  # Adjust based on the number of bits
#     # Create the window column by summing the bits within the window
#     raw_data[f'window{i}_{i + window_size - 1}'] = raw_data[[f'bit{j}' for j in range(i, i + window_size)]].sum(axis=1)

In [73]:
raw_data[["Num","window0_1","bit0", "bit1","bit2", "bit3"]]

,Num,window0_1,bit0,bit1,bit2,bit3
0,2,1,0,1,0,0
1,3,2,1,1,0,0
2,5,1,1,0,1,0
3,7,2,1,1,1,0
4,11,2,1,1,0,1
...,...,...,...,...,...,...
999994,15485807,2,1,1,1,1
999995,15485837,1,1,0,1,1
999996,15485843,2,1,1,0,0
999997,15485849,1,1,0,0,1


In [98]:
updated_data = raw_data

In [110]:
# Start the timer
start_time = time.time()

# Define parameters
window_size = 2
sums = [0, 1, 2]  # Possible sums for a 2-bit window
num_rows = len(updated_data)

# Process each window column
for i in range(20 - window_size + 1):  # For all 2-bit windows
    window_col = f'window{i}_{i+1}'

    # Extract the column as a NumPy array for faster processing
    window_values = updated_data[window_col].values

    # Initialize cumulative counts for each sum (0, 1, 2)
    cumulative_counts = np.zeros((num_rows, len(sums)))

    # Efficiently calculate cumulative counts using NumPy broadcasting
    for s in sums:
        # Create a mask for the current sum
        mask = (window_values == s).astype(int)
        cumulative_counts[:, s] = np.cumsum(mask)  # Cumulative count of this sum

    # Calculate cumulative probabilities for each sum
    total_counts = np.arange(1, num_rows + 1).reshape(-1, 1)  # Row indices as total count
    cumulative_probs = cumulative_counts / total_counts

    # Add cumulative probabilities as new columns in the dataset
    for idx, s in enumerate(sums):
        col_name = f'{window_col}_CumulativeP(Sum={s})'
        updated_data[col_name] = cumulative_probs[:, idx]

# End the timer
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Time spent on vectorized transformation: {elapsed_time:.2f} seconds")

Time spent on vectorized transformation: 1.09 seconds


In [111]:
updated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 98 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Num                             999999 non-null  int64  
 1   Interval                        999999 non-null  int64  
 2   bit19                           999999 non-null  int64  
 3   bit18                           999999 non-null  int64  
 4   bit17                           999999 non-null  int64  
 5   bit16                           999999 non-null  int64  
 6   bit15                           999999 non-null  int64  
 7   bit14                           999999 non-null  int64  
 8   bit13                           999999 non-null  int64  
 9   bit12                           999999 non-null  int64  
 10  bit11                           999999 non-null  int64  
 11  bit10                           999999 non-null  int64  
 12  bit9            

In [112]:
# Flatten the selection: single index, range [17:22], and range [41:53]
selected_columns = [0] + list(range(17, 22)) + list(range(41, 53))

# Select columns using the corrected list
subset_data = updated_data.iloc[:, selected_columns]
subset_data.head(16)

,Num,bit4,bit3,bit2,bit1,bit0,window0_1_CumulativeP(Sum=0),window0_1_CumulativeP(Sum=1),window0_1_CumulativeP(Sum=2),window1_2_CumulativeP(Sum=0),window1_2_CumulativeP(Sum=1),window1_2_CumulativeP(Sum=2),window2_3_CumulativeP(Sum=0),window2_3_CumulativeP(Sum=1),window2_3_CumulativeP(Sum=2),window3_4_CumulativeP(Sum=0),window3_4_CumulativeP(Sum=1),window3_4_CumulativeP(Sum=2)
0,2,0,0,0,1,0,0.0,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
1,3,0,0,0,1,1,0.0,0.500000,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,5,0,0,1,0,1,0.0,0.666667,0.333333,0.000000,1.000000,0.000000,0.666667,0.333333,0.000000,1.000000,0.000000,0.000000
3,7,0,0,1,1,1,0.0,0.500000,0.500000,0.000000,0.750000,0.250000,0.500000,0.500000,0.000000,1.000000,0.000000,0.000000
4,11,0,1,0,1,1,0.0,0.400000,0.600000,0.000000,0.800000,0.200000,0.400000,0.600000,0.000000,0.800000,0.200000,0.000000
5,13,0,1,1,0,1,0.0,0.500000,0.500000,0.000000,0.833333,0.166667,0.333333,0.500000,0.166667,0.666667,0.333333,0.000000
6,17,1,0,0,0,1,0.0,0.571429,0.428571,0.142857,0.714286,0.142857,0.428571,0.428571,0.142857,0.571429,0.428571,0.000000
7,19,1,0,0,1,1,0.0,0.500000,0.500000,0.125000,0.750000,0.125000,0.500000,0.375000,0.125000,0.500000,0.500000,0.000000
8,23,1,0,1,1,1,0.0,0.444444,0.555556,0.111111,0.666667,0.222222,0.444444,0.444444,0.111111,0.444444,0.555556,0.000000
9,29,1,1,1,0,1,0.0,0.500000,0.500000,0.100000,0.700000,0.200000,0.400000,0.400000,0.200000,0.400000,0.500000,0.100000


In [3]:
updated_data = pd.read_csv("../Data/updated_data.csv")

In [5]:
updated_data = updated_data[updated_data["Num"]<1048576]

In [4]:
updated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82025 entries, 0 to 82024
Data columns (total 98 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Num                             82025 non-null  int64  
 1   Interval                        82025 non-null  int64  
 2   bit19                           82025 non-null  int64  
 3   bit18                           82025 non-null  int64  
 4   bit17                           82025 non-null  int64  
 5   bit16                           82025 non-null  int64  
 6   bit15                           82025 non-null  int64  
 7   bit14                           82025 non-null  int64  
 8   bit13                           82025 non-null  int64  
 9   bit12                           82025 non-null  int64  
 10  bit11                           82025 non-null  int64  
 11  bit10                           82025 non-null  int64  
 12  bit9                            

In [5]:
dataBitProb = pd.read_csv("../Data/dataBitProb.csv")
dataWinProb = pd.read_csv("../Data/dataWinProb.csv")

In [7]:
dataWinProb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82025 entries, 0 to 82024
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Num                             82025 non-null  int64  
 1   Interval                        82025 non-null  int64  
 2   window0_1                       82025 non-null  int64  
 3   window1_2                       82025 non-null  int64  
 4   window2_3                       82025 non-null  int64  
 5   window3_4                       82025 non-null  int64  
 6   window4_5                       82025 non-null  int64  
 7   window5_6                       82025 non-null  int64  
 8   window6_7                       82025 non-null  int64  
 9   window7_8                       82025 non-null  int64  
 10  window8_9                       82025 non-null  int64  
 11  window9_10                      82025 non-null  int64  
 12  window10_11                     

In [10]:
# # Flatten the selection: single index, range [17:22], and range [41:53]
# selected_columns = [0] + [1] +list(range(22,98))

# # Select columns using the corrected list
# dataWinProb = updated_data.iloc[:, selected_columns]
# dataWinProb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82025 entries, 0 to 82024
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Num                             82025 non-null  int64  
 1   Interval                        82025 non-null  int64  
 2   window0_1                       82025 non-null  int64  
 3   window1_2                       82025 non-null  int64  
 4   window2_3                       82025 non-null  int64  
 5   window3_4                       82025 non-null  int64  
 6   window4_5                       82025 non-null  int64  
 7   window5_6                       82025 non-null  int64  
 8   window6_7                       82025 non-null  int64  
 9   window7_8                       82025 non-null  int64  
 10  window8_9                       82025 non-null  int64  
 11  window9_10                      82025 non-null  int64  
 12  window10_11                     82025

In [11]:
# # Flatten the selection: single index, range [17:22], and range [41:53]
# selected_columns = [0] + [1] +list(range(22,98))

# # Select columns using the corrected list
# dataBitProb = updated_data.iloc[:, selected_columns]

In [9]:
# updated_data.to_csv("../Data/updated_data.csv",index = False)

In [115]:
# raw_data.to_csv("../Data/1m.csv",index = False)

In [12]:
# dataWinProb.to_csv("../Data/dataWinProb.csv",index = False)

In [13]:
# dataBitProb.to_csv("../Data/dataBitProb.csv",index = False)